In [27]:
import numpy as np
import xarray as xr
import dask.array as da
import dask.dataframe as dd
import dask.bag as db


recordsarray = np.array([('Rex', 9, 81.0), ('Fido', 3, 27.0)], dtype=[('name', 'U10'), ('age', 'i4'), ('weight', 'f4')])
channelcoord = xr.DataArray(da.array(recordsarray), dims=["c"])
loaded = xr.DataArray(da.zeros((1024,1024,2,1)),dims=["x","y","c","z"], coords={"channels": channelcoord})

bag = db.from_sequence([loaded, loaded, loaded])

In [23]:
import dask

In [24]:
xr.__version__

'0.14.1'

In [28]:
def get_biometa(array):
            df = dd.from_dask_array(array.channels.data)
            df["Array"] = array.name
            return df

newbag = bag.map(get_biometa)

In [30]:
%%time
newbag.compute(scheduler="processes")

CPU times: user 33.4 ms, sys: 65.1 ms, total: 98.5 ms
Wall time: 143 ms


[Dask DataFrame Structure:
                  name    age   weight   Array
 npartitions=1                                
 0              object  int32  float32  object
 1                 ...    ...      ...     ...
 Dask Name: assign, 3 tasks, Dask DataFrame Structure:
                  name    age   weight   Array
 npartitions=1                                
 0              object  int32  float32  object
 1                 ...    ...      ...     ...
 Dask Name: assign, 3 tasks, Dask DataFrame Structure:
                  name    age   weight   Array
 npartitions=1                                
 0              object  int32  float32  object
 1                 ...    ...      ...     ...
 Dask Name: assign, 3 tasks]

In [15]:
class BioMetaAccesor(object):
    
    def __init__(self, bag):
        self.bag = bag
    
    @property
    def channels(self):
        
        def get_biometa(array):
            df = dd.from_dask_array(array.channels.data)
            df["Array"] = array.name
            return df
        
        datasets = self.bag.map(get_biometa)
        return datasets



class LarvikBag(object):

    def __init__(self, bag, *args, **kwargs):
        self.bag = bag
        self._biometa = None
        self.settings = {}

    def __getattr__(self, item):
        func = getattr(self.bag, item)
        def inner(*args, **kwargs):
            value = func(*args, **kwargs)
            if isinstance(value, Bag):
                return LarvikBag(value)
            else:
                return value

        return inner

    def parseSettings(self,settings):
        self.settings = settings

    def parse(self, *args):
        ''' :param *args Array of ParserSubclass'''
        parsedb = self.bag
        manager = LarvikManager()

        for i, parser in enumerate(args):
            parsedb = parsedb.map(parser.filter, self.settings, manager)

        return LarvikBag(parsedb)
    
    @property
    def biometa(self):
        if not self._biometa:
            self._biometa = BioMetaAccesor(self.bag)
        return self._biometa

    def __repr__(self):
        return self.bag.__repr__()

import dask.bag as db
bag =  LarvikBag(db.from_sequence([loaded, loaded, loaded]))

In [17]:
newbag = bag.map(get_biometa)

NameError: name 'Bag' is not defined